## Test Transits class
Using this file for testing and iteration. For all the examples, we should test modifying the prompt along with temperature and other properties to get a good output

In [341]:
from dotenv import load_dotenv
load_dotenv()  # fill out required environment variables in .env file

from openai import OpenAI
client = OpenAI()

## Helpers

In [342]:
import textwrap
import re


# Hard-coded mappings for demonstration
astrological_signs_mapped = {
    'Sun in Taurus': '[☉♉]',
    'Moon in Libra': '[☽♎]',
    'Mars in Capricorn': '[♂♑]',
    'Mercury in Aries': '[☿♈]',
    'Jupiter in Pisces': '[♃♓]',
    'Venus in Taurus': '[♀♉]',
    'Saturn in Sagittarius': '[♄♐]',
    'Uranus in Sagittarius': '[♅♐]',
    'Neptune in Capricorn': '[♆♑]',
    'Pluto in Scorpio': '[♇♏]',
    'Moon in Gemini': '[☽♊]',
    'Sun in Aquarius': '[☉♒]',
    'Mercury in Aquarius': '[☿♒]',
    'Venus in Aquarius': '[♀♒]',
    'Mars in Aquarius': '[♂♒]',
    'Jupiter in Taurus': '[♃♉]',
    'Neptune in Pisces': '[♆♓]',
    'Saturn in Pisces': '[♄♓]',
    'Uranus in Taurus': '[♅♉]'
}


def format_completion(completion, signs_mapped=astrological_signs_mapped):
    # Assuming 'completion' can be a string or an object with the necessary attribute
    if hasattr(completion, 'choices'):
        content = completion.choices[0].message.content.strip()
    else:
        content = completion
    
    # Function to replace astrological signs with their references in a given text
    def replace_signs_with_references(text, signs_mapped):
        for sign, reference in signs_mapped.items():
            pattern = r'\b' + re.escape(sign) + r'\b'
            text = re.sub(pattern, f"{sign}{reference}", text)
        return text
    
    # Apply the replacements to the content
    content_with_references = replace_signs_with_references(content, signs_mapped)
    
    # Split the content into paragraphs
    paragraphs = content_with_references.split('\n\n')
    
    # Wrap each paragraph and join with double newlines
    wrapped_paragraphs = '\n\n'.join(['\n'.join(textwrap.wrap(paragraph, width=80)) for paragraph in paragraphs])
    
    return wrapped_paragraphs
    

# Example completion (this would be replaced with the actual completion object)
completion_example = "Considering your Sun in Taurus and the current transit of Mars in Aquarius, it might be a time for reevaluation rather than abrupt changes..."

# Call the function with an example completion and the mappings
print(format_completion(completion_example, signs_mapped=astrological_signs_mapped))


Considering your Sun in Taurus[☉♉] and the current transit of Mars in
Aquarius[♂♒], it might be a time for reevaluation rather than abrupt changes...


## With detailed transit context
In addition to providing transits, the llm may have a better output if we include traits of those transits

In [343]:
astrological_transits_context = """
- The collective energy of Sun, Mercury, Venus, and Mars in Aquarius brings a focus on innovation, social justice, and community building.
- Jupiter in Taurus emphasizes growth in financial matters and personal values.
- Saturn and Neptune in Pisces highlight spiritual growth, creative expression, and facing illusions.
"""

personal_birth_signs = """
- Sun in Taurus: Your core being shines through persistence, sensuality, and an appreciation for the finer things in life.
- Moon in Libra: You have an innate need for harmony, balance, and beautiful surroundings, influencing your emotional responses.
- Venus in Taurus: This placement enhances your love for beauty, stability in relationships, and a strong connection to nature.
- Mars in Capricorn: It gives you a disciplined, structured approach to pursuing your goals and ambitions.
"""

prompt = f"""Directly provide a personalized and engaging horoscope without any greetings or closures. Focus on the content based on the current astrological transits and my specific birth signs. 2 paragraphs
Transits: {astrological_transits_context}
My Birth Signs: {personal_birth_signs}
"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt},
    ]
)

print(format_completion(completion))


With the current cosmic energies at play, your focus on innovation and community
building is heightened. Embrace the Aquarian energy to bring about positive
changes in social justice issues and collaborative endeavors. Jupiter's
influence in Taurus prompts growth in your financial matters and values, urging
you to expand your perspectives and cultivate abundance in new ways. Saturn and
Neptune in Pisces[♆♓] encourage you to explore spiritual growth and creative
expression, while also pushing you to confront any illusions or self-deceptions
that may be holding you back.

Your Taurus sun sign, combined with a Libra moon, suggests a unique blend of
persistence and harmony in your emotional responses. Venus in Taurus[♀♉]
enhances your love for beauty and stability in relationships, guiding you to
deepen your connections with those around you. With Mars in Capricorn[♂♑], your
disciplined approach to pursuing goals will serve you well during this time of
growth and expansion. Embrace these i

## Use all Transits and Birth Signs
In this test I'm trying to see if the model can take in a list of all transits and birth signs, and formulate a good output for the question

In [344]:
user_question = "How do I brighten someone's day?"
user_birth_signs = ["Sun in Taurus", "Moon in Libra", "Mars in Capricorn", "Mercury in Aries", "Jupiter in Pisces", "Venus in Taurus", "Saturn in Sagittarius", "Uranus in Sagittarius", "Neptune in Capricorn", "Pluto in Scorpio"]
current_transits = ["Moon in Gemini", "Sun in Aquarius", "Mercury in Aquarius", "Venus in Aquarius", "Mars in Aquarius", "Jupiter in Taurus", "Neptune in Pisces", "Saturn in Pisces", "Uranus in Taurus"]

selected_birth_signs = user_birth_signs
selected_transits = current_transits

system_message = {
    "role": "system",
    "content": (
        f"Provide thoughtful astrological advice considering two of the user's birth signs ({', '.join(selected_birth_signs)}) and "
        f"two of the current transits ({', '.join(selected_transits)}). Consider how these astrological factors might influence the user's question, "
        f"using empathy and practical guidance. Vary which transits and birth signs are selected for each message."
        f"Your inital response should be approximately 150 to 250 words in length."
    )
}

# Send the user's question and system message to the API
user_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": user_question},
        system_message
    ],
    temperature=1,
    stream=True  # Set stream=True to receive responses incrementally
)

print(f"User question: {user_question}\n")

response_string = f"""User question: {user_question}

"""
for chunk in user_response:
    if chunk.choices and chunk.choices[0].delta.content is not None:
        response_string += chunk.choices[0].delta.content
        formatted_response = format_completion(response_string)
        clear_output(wait=False)  # Clear the output before updating
        print(formatted_response,  end='', flush=True)  # Print the formatted response

clear_output(wait=False)  # Clear the output before updating
print(formatted_response,  end='', flush=True)  # Print the formatted response

User question: How do I brighten someone's day?

Today, with the Moon in Gemini[☽♊] and Mercury in Aquarius[☿♒], communication
and intellectual stimulation are key themes. With your Sun in Taurus[☉♉] and
Venus in Taurus[♀♉], you can bring warmth and stability to those around you.
Maybe reach out to a friend or loved one with a thoughtful message or a small
gesture that shows you care. Your innate sense of beauty and harmony, with your
Moon in Libra[☽♎], can bring a touch of elegance and grace to someone's day.

Mars in Capricorn[♂♑] and Saturn in Sagittarius[♄♐] give you determination and a
sense of responsibility, so offering practical help or solutions to someone in
need can make a real difference. Remember that your Jupiter in Pisces[♃♓]
enhances your empathy and compassion, allowing you to connect on a deeper level
with others.

Overall, by combining your practicality, warmth, and empathy, you can brighten
someone's day by being there for them, offering a listening ear, and providi

## Follow up questions
Figuring out how to get the model good at responding to follow up questions

In [345]:
user_question = "I'm also thinking about quitting my job. What do you think?"
user_birth_signs = ["Sun in Taurus", "Moon in Libra", "Mars in Capricorn", "Mercury in Aries", "Jupiter in Pisces", "Venus in Taurus", "Saturn in Sagittarius", "Uranus in Sagittarius", "Neptune in Capricorn", "Pluto in Scorpio"]
current_transits = ["Moon in Gemini", "Sun in Aquarius", "Mercury in Aquarius", "Venus in Aquarius", "Mars in Aquarius", "Jupiter in Taurus", "Neptune in Pisces", "Saturn in Pisces", "Uranus in Taurus"]

selected_birth_signs = user_birth_signs
selected_transits = current_transits

system_message = {
    "role": "system",
    "content": (
        f"Provide thoughtful astrological advice considering two of the user's birth signs ({', '.join(selected_birth_signs)}) and "
        f"two of the current transits ({', '.join(selected_transits)}). Consider how these astrological factors might influence the user's question, "
        f"using empathy and practical guidance. Vary which transits and birth signs are selected for each message."
        f"Your inital response should be approximately 150 to 250 words in length. For the rest of the conversation, be more concise, with as few words as 1 unless the user asks for more details."
    )
}

# Send the user's question and system message to the API
user_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        system_message,
        {"role": "user", "content": "Should I invite my family over? They drive me crazy"},
        {"role": "assistant", "content":"Considering that you are a Taurus with a Moon in Libra, you likely value\nstability and harmony in your relationships. However, with the current transit\nof the Moon in Gemini, you may feel more inclined to socialize and connect\nwith your family despite any potential conflicts.\nWhile your family may drive you crazy at times, it could be beneficial to set\nclear boundaries and expectations before inviting them over. With Venus,\nMercury, and Mars all in Aquarius, you may find that open communication and a\nrational approach to addressing any issues can lead to a more positive\ngathering.\nAdditionally, with Jupiter in Taurus, this is a favorable time for expanding\nyour social circle and strengthening family bonds. It may be worth considering\nthe potential benefits of spending quality time with your family, even if they\ncan be challenging at times.\nUltimately, the decision to invite your family over should align with your\npersonal values and well-being. Trust your intuition and consider how you can\ncreate a harmonious and enjoyable environment for everyone involved.Considering that you are a Taurus"},
        {"role": "user", "content": user_question},
    ],
    temperature=.5,
    stream=True  # Set stream=True to receive responses incrementally
)

response_string = f"""User question: {user_question}

"""
for chunk in user_response:
    if chunk.choices and chunk.choices[0].delta.content is not None:
        response_string += chunk.choices[0].delta.content
        formatted_response = format_completion(response_string)
        clear_output(wait=False)  # Clear the output before updating
        print(formatted_response,  end='', flush=True)  # Print the formatted response


clear_output(wait=False)  # Clear the output before updating
print(formatted_response,  end='', flush=True)  # Print the formatted response

User question: I'm also thinking about quitting my job. What do you think?

With your Mars in Capricorn[♂♑] and Saturn in Sagittarius[♄♐], you likely
approach work with determination and a sense of responsibility. However, the
current transits of Saturn and Neptune in Pisces[♆♓] may be influencing your
desire for a change in your career path. Before making any decisions, consider
how this potential change aligns with your long-term goals and financial
stability. With Jupiter in Taurus[♃♉], this could be a favorable time to explore
new opportunities that resonate with your values and bring growth. Reflect on
whether your current job aligns with your passions and if there are ways to make
it more fulfilling before deciding to quit. With Uranus in Taurus[♅♉], you may
be seeking more freedom and innovation in your work environment. Take time to
assess your options, weigh the pros and cons, and consider speaking with a
trusted mentor or career advisor for guidance. Remember to trust your in